In [ ]:
import zarr
from brainlit.algorithms.generate_fragments.state_generation import state_generation
from pathlib import Path
import numpy as np
import napari

## Read Coordinates

In [ ]:
root_dir = Path(os.path.abspath('')).parents[0]
data_dir = os.path.join(root_dir, "data", "example")
im_path = os.path.join(data_dir, "image.zarr")
probs_path = os.path.join(data_dir, "probabilities.zarr")
coords_path = os.path.join(data_dir, "manual_coords.csv")

im = zarr.open(im_path)

coords = np.genfromtxt(coords_path, delimiter=',')
coords = coords.astype(int)
coords_list = list(coords)
coords_list = [list(c) for c in coords_list]
coords_list.reverse()
print(f"coords shape: {coords.shape}")
soma_coords = [list(coords[0,:])]
axon_coords = [list(coords[-1,:])]

## Process states

In [ ]:
sg = state_generation(image_path=im_path,
        ilastik_program_path=None,
        ilastik_project_path=None,
        chunk_size=[259, 443, 112],
        soma_coords=soma_coords,
        prob_path=probs_path)

sg.compute_frags()
sg.compute_soma_lbls()
sg.compute_image_tiered()
sg.compute_states()
sg.compute_edge_weights()
viterbrain = sg.viterbrain

## Compute shortest path

In [ ]:
vb_path = viterbrain.shortest_path(axon_coords[0], soma_coords[0])
vb_path = [list(coord) for coord in vb_path]

## Visualize

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im)
viewer.add_shapes(vb_path, shape_type='path', edge_width=2, edge_color='red', name="viterbrain")
viewer.add_shapes(coords_list, shape_type='path', edge_width=2, edge_color='green', name="truth")